In [54]:
import torch
import hydra

from train import train
from dataset_temp import MyDataset
from models.base_model import MyModel
from torch.utils.data import DataLoader
from utils import main_utils, train_utils#, data_loader
from utils.train_logger import TrainLogger
from omegaconf import DictConfig, OmegaConf
import os

import pickle
import h5py
import time
import torch
import torch.nn as nn

from tqdm import tqdm
from utils.types import Scores, Metrics
from utils.train_utils import TrainParams
from utils.train_logger import TrainLogger
import torch.nn.functional as F


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
with open("../data/cache/trainval_ans2label.pkl", "rb") as f:
    unpickler = pickle.Unpickler(f)
    dict_answers = unpickler.load()
    print(len(dict_answers))
    #self.number_of_answers_per_question = num_of_answers
#print("self.number_of_answers_per_question = ", self.number_of_answers_per_question)


1073


In [7]:
questions_train = torch.load("../data/questions_train")
questions_val = torch.load("../data/questions_val")

In [10]:
with open("../data/questions_dict_train", 'rb') as handle:
    questions_dict_train = pickle.load(handle)
with open("../data/questions_dict_val", 'rb') as handle:
    questions_dict_val = pickle.load(handle)


In [15]:
import json
with open("../data/question_id_to_image_id_train", 'r') as fd:
    question_id_to_image_id_train = json.load(fd)
with open("../data/question_id_to_image_id_val", 'r') as fd:
    question_id_to_image_id_val = json.load(fd)

In [23]:
with open("../data/answerable_with_labels_only_train_False", 'rb') as handle:
    answerable_train = pickle.load(handle)
with open("../data/answerable_with_labels_only_val_False", 'rb') as handle:
    answerable_val = pickle.load(handle)

In [32]:
print(len(answerable_val))

214354


In [55]:
torch.backends.cudnn.benchmark = True


print("current dict = ", os.getcwd())
"""
Run the code following a given configuration
:param cfg: configuration file retrieved from hydra framework
"""
# # main_utils.init(cfg)
# logger = TrainLogger(exp_name_prefix=cfg['main']['experiment_name_prefix'], logs_dir=cfg['main']['paths']['logs'])
# print(logger)
# logger.write(OmegaConf.to_yaml(cfg))

# # Set seed for results reproduction
# main_utils.set_seed(cfg['main']['seed'])

# Load dataset
train_dataset = MyDataset(image_path='../../../datashare/train2014',
                          questions_path='../../../datashare/v2_OpenEnded_mscoco_train2014_questions.json',
                          answers_path='../../../datashare/v2_mscoco_train2014_annotations.json',
                          train=True,
                          answerable_only = False
                         )
# val_dataset = MyDataset(image_path='../../../datashare/val2014',
#                           questions_path='../../../datashare/v2_OpenEnded_mscoco_val2014_questions.json',
#                           answers_path='../../../datashare/v2_mscoco_val2014_annotations.json',
#                           train=False,
#                           answerable_only = False
#                          )


train_loader = DataLoader(train_dataset, 128, shuffle=True,
                          num_workers=8)
# eval_loader = DataLoader(val_dataset, 128, shuffle=False,
#                          num_workers=8)
 


# Init model
image_in_size_input = ((3,224,224))
img_encoder_channels_input = [32, 128, 512, 1024]
# model = MyModel(image_in_size=image_in_size_input,
#     img_encoder_out_classes=1024,
#     img_encoder_channels=img_encoder_channels_input,
#     img_encoder_batchnorm=True,
#     img_encoder_dropout=0.5,
#     text_embedding_tokens=15193,
#     text_embedding_features=100,
#     text_lstm_features=512,
#     text_dropout=0.5,
#     attention_mid_features=128,
#     attention_glimpses=2,
#     attention_dropout=0.5,
#     classifier_dropout=0.5,
#     classifier_mid_features=128,
#     classifier_out_classes=2410,
#     )



# if cfg['main']['parallel']:
#     model = torch.nn.DataParallel(model)

# if torch.cuda.is_available():
# torch.cuda.empty_cache()
# model = model.cuda()

# logger.write(main_utils.get_model_string(model))

# Run model
# train_params = train_utils.get_train_params(cfg)

# Report metrics and hyper parameters to tensorboard


# metrics = train(model, train_loader, eval_loader, train_params, logger)
# hyper_parameters = main_utils.get_flatten_dict(cfg['train'])

# logger.report_metrics_hyper_params(hyper_parameters, metrics)

current dict =  /home/student/DL_hw2
files upload was done
opening existing Qs
questions done
question_id_to_image_id done
answers done
images done
coco_images_to_dict done


In [45]:
def __apply_padding( question):
    if len(question) < 22:
        padding = [0] * (22 - len(question))
        question.extend(padding)
    return question

In [60]:
train_dataset.target[1]

{'question_id': 458752001,
 'question_type': 'what',
 'question_idx': [2,
  10,
  3,
  4,
  11,
  12,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'image_id': 458752,
 'label_counts': {1: 9, 66: 1},
 'labels': [1, 66],
 'scores': [1, 0.3]}

In [56]:
item = 1
target = train_dataset.target[item]
q_id = target['question_id']

        answer_labels = target['labels']
        answer_scores = target['scores']

        # a = train_dataset.answerable[q_id]
        ans = torch.zeros(train_dataset.number_of_answers_per_question)
        for answer_index in range(len(answer_labels)):
            ans[answer_labels[answer_index]] = answer_scores[answer_index]
image_id = train_dataset.question_id_to_image_id[str(q_id)]
images = h5py.File("../data/cache/train.h5", 'r')
image_index = train_dataset.img2idx[image_id]
v = images['images'][image_index].astype('float32')
v = torch.from_numpy(v)
question = target['question_idx']
question = __apply_padding(question)
question = torch.tensor(question, dtype=torch.long)


In [59]:
train_dataset.answerable


{458752000: ([0], [1]),
 458752001: ([1, 61], [1, 0.3]),
 458752002: ([2], [1]),
 458752003: ([3, 9], [1, 0.3]),
 262146000: ([4], [1]),
 262146001: ([5], [1]),
 262146002: ([6, 28, 69], [1, 0.3, 0.3]),
 524291000: ([7], [1]),
 524291001: ([3], [1]),
 524291002: ([7], [1]),
 393221000: ([3], [1]),
 393221001: ([3], [1]),
 393221002: ([341, 201], [0.3, 0.3]),
 393223000: ([3, 9], [1, 0.6]),
 393223001: ([4], [0.9]),
 393223002: ([8, 486], [1, 0.3]),
 393223003: ([3], [1]),
 393224000: ([9], [1]),
 393224001: ([1063, 184, 136, 424], [0.3, 0.3, 0.6, 0.3]),
 393224002: ([3, 9], [0.9, 1]),
 393224003: ([9, 3], [1, 0.3]),
 393224004: ([9], [1]),
 393224005: ([9, 3], [1, 0.3]),
 524297000: ([3, 9], [1, 0.9]),
 524297001: ([10, 647], [1, 0.3]),
 524297002: ([9, 3], [1, 0.3]),
 524297003: ([9], [1]),
 393227000: ([3], [1]),
 393227001: ([11], [1]),
 393227002: ([12, 30, 17], [1, 0.3, 0.3]),
 393227003: ([13], [1]),
 393227004: ([9], [1]),
 131084000: ([14], [1]),
 131084001: ([3], [1]),
 131084

In [51]:
ans

tensor([0., 1., 0.,  ..., 0., 0., 0.])

In [17]:
import json
with open('../../../datashare/v2_OpenEnded_mscoco_val2014_questions.json', 'r') as fd:
    questions_json = json.load(fd)

In [8]:
with open('../data/cache/train_target.pkl', "rb") as f:
    target = pickle.load(f)

In [10]:
target

[{'question_id': 458752000,
  'question_type': 'what is this',
  'question_idx': [2, 3, 4, 5, 6, 7, 8, 9],
  'image_id': 458752,
  'label_counts': {0: 8},
  'labels': [0],
  'scores': [1]},
 {'question_id': 458752001,
  'question_type': 'what',
  'question_idx': [2, 10, 3, 4, 11, 12, 9],
  'image_id': 458752,
  'label_counts': {1: 9, 66: 1},
  'labels': [1, 66],
  'scores': [1, 0.3]},
 {'question_id': 458752002,
  'question_type': 'what color is the',
  'question_idx': [2, 13, 3, 14, 15, 9],
  'image_id': 458752,
  'label_counts': {2: 10},
  'labels': [2],
  'scores': [1]},
 {'question_id': 458752003,
  'question_type': 'is this',
  'question_idx': [3, 4, 11, 16, 17, 18, 9],
  'image_id': 458752,
  'label_counts': {3: 9, 9: 1},
  'labels': [3, 9],
  'scores': [1, 0.3]},
 {'question_id': 262146000,
  'question_type': 'what color is the',
  'question_idx': [2, 13, 3, 19, 9],
  'image_id': 262146,
  'label_counts': {4: 10},
  'labels': [4],
  'scores': [1]},
 {'question_id': 262146001,
  

In [19]:
questions = [q['question'] for q in questions_json['questions']]

In [20]:
questions

['Where is he looking?',
 'What are the people in the background doing?',
 'What is he on top of?',
 'What website copyrighted the picture?',
 'Is this a creamy soup?',
 'Is this rice noodle soup?',
 'What is to the right of the soup?',
 'What is the man doing in the street?',
 "How many photo's can you see?",
 'What does the truck on the left sell?',
 'Why is there a gap between the roof and wall?',
 'Is it daylight in this picture?',
 'Why is the cow laying down?',
 'What color is the grass in this picture?',
 'Did the batter hit the ball?',
 'How many are playing ball?',
 'Is there a chain link fence in the image?',
 'Is the boy playing baseball?',
 'Is that a folding chair?',
 'How many beds?',
 'What color is the bedspread?',
 'How many pictures are there?',
 'Are these twin mattresses?',
 'How many sources of light are there?',
 'Is this room decorated for the 1970s?',
 'Are the lights on in this room?',
 'Are the windows big?',
 'What is the chair made of?',
 "Is this room in so

In [21]:
for question in questions:
    print(question)
    question = question.lower()[:-1]
    question = question.split(' ')
    print(question)
    break

Where is he looking?
['where', 'is', 'he', 'looking']


In [11]:
questions = torch.load("../data/questions_train")


In [30]:
question = questions[0]

In [48]:
with open('../data/cache/train_target.pkl', "rb") as f:
    target_all = pickle.load(f)


In [49]:
target_all[1]

{'question_id': 458752001,
 'question_type': 'what',
 'question_idx': [2, 10, 3, 4, 11, 12, 9],
 'image_id': 458752,
 'label_counts': {1: 9, 66: 1},
 'labels': [1, 66],
 'scores': [1, 0.3]}

In [6]:
print(vocab_json)

{'PAD': 0, 'UNK': 1, 'what': 2, 'is': 3, 'this': 4, 'photo': 5, 'taken': 6, 'looking': 7, 'through': 8, '?': 9, 'position': 10, 'man': 11, 'playing': 12, 'color': 13, 'players': 14, 'shirt': 15, 'professional': 16, 'baseball': 17, 'player': 18, 'snow': 19, 'person': 20, 'doing': 21, 'persons': 22, 'headwear': 23, 'in': 24, "person's": 25, 'hand': 26, 'dog': 27, 'waiting': 28, 'at': 29, 'tennis': 30, 'ball': 31, 'or': 32, 'frisbee': 33, 'sky': 34, 'blue': 35, 'there': 36, 'on': 37, 'mountains': 38, 'white': 39, 'streak': 40, 'window': 41, 'open': 42, 'toothbrush': 43, 'child': 44, 'she': 45, 'brushing': 46, 'smiling': 47, 'business': 48, 'picture': 49, 'does': 50, 'his': 51, 'tie': 52, 'pair': 53, 'well': 54, 'with': 55, 'suit': 56, 'look': 57, 'happy': 58, 'wearing': 59, 'plain': 60, 'judging': 61, 'from': 62, 'dress': 63, ',': 64, 'was': 65, 'latin': 66, 'american': 67, 'country': 68, 'colors': 69, 'are': 70, 'shown': 71, 'men': 72, 'sidewalk': 73, 'recently': 74, 'guy': 75, 'have': 7

In [86]:
_, y_hat = a.max(dim=1, keepdim=True)
answer = torch.zeros(a.shape)
for i in range(len(y_hat)):
    answer[i][y_hat[i]] = 1
#     answer[i][y_hat[i]+1]= 1
def answer_norm(ans):
    zeros = torch.zeros(ans.shape[1])
    zeros = zeros .cuda()
    max_values, predicted_index = ans.max(dim=1, keepdim=True)
    for i in range(ans.shape[0]):
        ans[i] = torch.where(ans[i]==max_values[i], ans[i], zeros)
    ans_sum = ans.sum(dim=1)
    for i in range(len(ans_sum)):
        if ans_sum[i] == 0:
            ans_sum[i]=1
        ans[i] = ans[i]/ans_sum[i]
    return ans

def batch_accuracy(predicted, true):
    _, predicted_index = predicted.max(dim=1, keepdim=True)
    agreeing = true.gather(dim=1, index=predicted_index)
    return (agreeing)


answer = answer.cuda()
a = a.cuda()
score = 0
score += train_utils.batch_accuracy(answer, a.data).sum()
print(score)
for i in range(10):
    for j in range(a.shape[1]):
        if a[i][j] > 0:
            print("i = ", i, "j = ", j, "value = ",a[i][j])
a = answer_norm(a)
for i in range(10):
    for j in range(a.shape[1]):
        if a[i][j] > 0:
            print("i = ", i, "j = ", j, "value = ",a[i][j])
loss_func = nn.LogSoftmax(dim=1).to("cuda")
nll = -loss_func(answer)
loss = (nll * ans).sum(dim=1).mean()
print(loss)


tensor(118.3000, device='cuda:0')
i =  0 j =  11 value =  tensor(0.3000, device='cuda:0')
i =  0 j =  74 value =  tensor(0.3000, device='cuda:0')
i =  0 j =  226 value =  tensor(1., device='cuda:0')
i =  1 j =  898 value =  tensor(1., device='cuda:0')
i =  2 j =  11 value =  tensor(0.3000, device='cuda:0')
i =  2 j =  74 value =  tensor(0.9000, device='cuda:0')
i =  4 j =  9 value =  tensor(1., device='cuda:0')
i =  5 j =  3 value =  tensor(1., device='cuda:0')
i =  6 j =  362 value =  tensor(0.3000, device='cuda:0')
i =  6 j =  530 value =  tensor(1., device='cuda:0')
i =  7 j =  136 value =  tensor(1., device='cuda:0')
i =  8 j =  12 value =  tensor(1., device='cuda:0')
i =  8 j =  49 value =  tensor(1., device='cuda:0')
i =  9 j =  865 value =  tensor(1., device='cuda:0')
i =  0 j =  226 value =  tensor(1., device='cuda:0')
i =  1 j =  898 value =  tensor(1., device='cuda:0')
i =  2 j =  74 value =  tensor(1., device='cuda:0')
i =  4 j =  9 value =  tensor(1., device='cuda:0')
i =  

In [87]:
score
    

tensor(118.3000, device='cuda:0')

In [88]:
len(eval_loader.dataset)

214354

In [14]:
# # temp = {}
# id_and_extension = temp.split('_')[-1]
# id = int(id_and_extension.split('.')[0])
# print(id)

42


In [89]:
class QuestionNet(nn.Module):
    def __init__(self, word_embedding_dim, lstm_hidden_dim, word_vocab_size, output_dim, lstm_drop=0.0):
        super(QuestionNet, self).__init__()

        # LSTM unit
        self.word_embedding = nn.Embedding(word_vocab_size, word_embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(p=lstm_drop)
        self.lstm_input_dim = self.word_embedding.embedding_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        self.lstm = nn.LSTM(input_size=self.lstm_input_dim, hidden_size=self.lstm_hidden_dim, num_layers=2)
        self.fc = nn.Linear(output_dim, output_dim, bias=True)
        # self.fc2 = nn.Linear(21 * self.lstm_hidden_dim, q_output_dim, bias=True)
        # self.output = q_output_dim


    def forward(self, questions_tensor):
        t = self.word_embedding(questions_tensor)
        t = self.dropout(t)
        t = F.tanh(t)
        t = t.transpose(0, 1)
        _, (h, c) = self.lstm(t)
        t = torch.cat((h, c), 2)
        t = t.transpose(0, 1)
        t = t.reshape(t.size()[0], -1)
        t = F.tanh(t)
        return self.fc(t)


In [93]:
QP = QuestionNet(300, 1024, 15193, 4096, 0.5)


In [94]:
matan = QP(q)

/home/student/.conda/envs/cs236781-hw/lib/python3.8/site-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [100]:
example = torch.ones(2,3)
example[1][2] = 3

In [97]:
temp = torch.mul(matan, matan)

In [101]:
temp = torch.mul(example, example)

In [111]:
with open("../data/cache/train_target.pkl", "rb") as f:
    unpickler = pickle.Unpickler(f)
    scores = unpickler.load()


In [112]:
print(len(scores))

443757


In [115]:
scores

[{'question_id': 458752000,
  'question_type': 'what is this',
  'image_id': 458752,
  'label_counts': {0: 8},
  'labels': [0],
  'scores': [1]},
 {'question_id': 458752001,
  'question_type': 'what',
  'image_id': 458752,
  'label_counts': {1: 9, 61: 1},
  'labels': [1, 61],
  'scores': [1, 0.3]},
 {'question_id': 458752002,
  'question_type': 'what color is the',
  'image_id': 458752,
  'label_counts': {2: 10},
  'labels': [2],
  'scores': [1]},
 {'question_id': 458752003,
  'question_type': 'is this',
  'image_id': 458752,
  'label_counts': {3: 9, 9: 1},
  'labels': [3, 9],
  'scores': [1, 0.3]},
 {'question_id': 262146000,
  'question_type': 'what color is the',
  'image_id': 262146,
  'label_counts': {4: 10},
  'labels': [4],
  'scores': [1]},
 {'question_id': 262146001,
  'question_type': 'what is the person',
  'image_id': 262146,
  'label_counts': {5: 10},
  'labels': [5],
  'scores': [1]},
 {'question_id': 262146002,
  'question_type': 'what color is the',
  'image_id': 262146

In [113]:
scores = [t for t in scores if t['label_counts']]

In [23]:
with open("../data/cache/trainval_q2label.pkl", "rb") as f:
    unpickler = pickle.Unpickler(f)
    vocab_json = unpickler.load()
vocab = vocab_json
token_to_index = vocab#['question']


In [25]:
len(token_to_index)

13659

In [31]:
""" Tokenize and normalize questions from a given question json in the usual VQA format. """
questions = [q['question'] for q in self.questions_json['questions']]
for question in questions:
    question = question.lower()[:-1]
    yield question.split(' ')


'Where is he looking?'

In [34]:
vec = torch.zeros(22).long()
for i, token in enumerate(question):
    print(token)
    index = token_to_index.get(token, 0)
    vec[i] = index


W
h
e
r
e
 
i
s
 
h
e
 
l
o
o
k
i
n
g
?


In [33]:
vec

tensor([    0, 11250,  4987,  7554,  4987,     0,  1293,  2830,     0, 11250,
         4987,     0,  6494,  5348,  5348,  6619,  1293,  8670, 10156,     9,
            0,     0])

In [104]:
#load question vocab
with open("../data/cache/question_vocab_train", 'r') as fd:
    vocab_json_train = json.load(fd)
with open("../data/cache/question_vocab_val", 'r') as fd:
    vocab_json_val = json.load(fd)


# #Vocab
# self.vocab = vocab_json
# self.token_to_index = self.vocab#['question']


In [106]:
print(len(vocab_json_train))
print(len(vocab_json_val))

15192
11467


In [5]:
#load the dataset of I, Q, A including the vocab of Q and A
with open('../../../datashare/v2_OpenEnded_mscoco_val2014_questions.json', 'r') as fd:
    questions_json = json.load(fd)
dataset_type = "val"

#load question vocab
with open("../data/cache/question_vocab_"+dataset_type, 'r') as fd:
    vocab_json = json.load(fd)

#Vocab
vocab = vocab_json
token_to_index = vocab

with open("../data/cache/trainval_ans2label.pkl", "rb") as f:
    unpickler = pickle.Unpickler(f)
    # if file is not empty scores will be equal
    # to the value unpickled
    dict_answers = unpickler.load()
    number_of_answers_per_question = len(dict_answers)


print("files upload was done")

#load Q
questions = torch.load("../data/questions_"+dataset_type)
print("questions done")

#change Q to Q dict    
with open("../data/questions_dict_"+dataset_type, 'rb') as handle:
    questions_dict = pickle.load(handle)

print("questions dict done")

#Load question_id_to_image_id
with open("../data/question_id_to_image_id_"+dataset_type, 'r') as fd:
    question_id_to_image_id = json.load(fd)

print("question_id_to_image_id done")

answerable_only=False
#load A
with open("../data/answerable_with_labels_only_"+dataset_type+"_"+str(answerable_only), 'rb') as handle:
    answerable = pickle.load(handle)

print("answers done")

with open("../data/cache/img2idx_"+dataset_type+".pkl", 'rb') as handle:
    img2idx = pickle.load(handle)
print("images done")

#load coco_images_to_dict
with open("../data/coco_images_to_dict"+dataset_type, 'rb') as handle:
    images_dict = pickle.load(handle)

print("coco_images_to_dict done")

def index_to_question_number_func(answerable):
    index_to_question_number_dict = {}
    cnt = 0
    for question in answerable:
        index_to_question_number_dict[cnt] = question
        cnt += 1
    return index_to_question_number_dict

index_to_question_number_dict = index_to_question_number_func(answerable)

files upload was done
questions done
questions dict done
question_id_to_image_id done
answers done
images done
coco_images_to_dict done


In [6]:
images = h5py.File("../data/cache/"+dataset_type+".h5", 'r')
with open("../data/cache/img2idx_"+dataset_type+".pkl", 'rb') as handle:
    img2idx = pickle.load(handle)

In [7]:
item = index_to_question_number_dict[1]
q, q_length = questions_dict[item]
a = answerable[item]
temp = torch.zeros(2410)
for answer_index in range(len(a[0])):
    temp[a[0][answer_index]] = a[1][answer_index]
print("item = ", item)
image_id = question_id_to_image_id[str(item)]
print("image_id = ", image_id)
image_index = img2idx[image_id]
print("image_index = ", image_index)
v = images['images'][image_index].astype('float16')
v = torch.from_numpy(v)

item =  262148001
image_id =  262148
image_index =  18264


KeyError: "Unable to open object (object 'images' doesn't exist)"

In [34]:
len(index_to_question_number_dict)

214354

In [35]:
len(questions_dict)

214354

In [36]:
len(answerable)

214354

In [37]:
len(question_id_to_image_id)

214354

In [38]:
len(img2idx)

24007

In [41]:
len(images['images'])

214354

In [43]:
images['images']

<HDF5 dataset "images": shape (214354, 3, 224, 224), type "<f2">

In [44]:
with open('../data/cache/val_target.pkl', "rb") as f:
    target = pickle.load(f)

In [ ]:
target

In [48]:
dataset_type = "train"
with open("../data/images_"+dataset_type, 'rb') as handle:
    images = pickle.load(handle)


In [51]:
len(images[0])

82783

In [59]:
for i in index_to_question_number_dict:
    temp = index_to_question_number_dict[i]
    temp2 = question_id_to_image_id[str(temp)]
    if(temp2 == 393225):
        print(temp)
    temp3 = img2idx[temp2]

393225000


KeyError: 393225

In [54]:
img2idx[393225]

KeyError: 393225

In [61]:
question_id_to_image_id["393225000"]

393225